In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('apex_model_trt.pth'))

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar(
    steering_gain=0.7,
    steering_offset=0.0,
    throttle_gain=-1.0,
    throttle_offset=0.0
)

In [ ]:
from IPython.display import display
import ipywidgets

steering_gain = ipywidgets.FloatSlider(min=0.0, max=3.0, value=0.7, step=0.001, description='steering gain')
steering_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='steering offset')
log_button = ipywidgets.ToggleButton(value=False, description='enable logging')

display(
    steering_gain,
    steering_offset,
    log_button
)

In [ ]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=60)

camera.running = True

In [ ]:
car.throttle = 0.06

In [ ]:
import cv2
import os
from utils import preprocess

LOG_DIRECTORY = 'circuit_log'

if not os.path.exists(LOG_DIRECTORY):
    os.mkdir(LOG_DIRECTORY)
    
count = 0

def execute(change):
    global count, LOG_DIRECTORY
    image = change['new']
    data = preprocess(image)
    xy = model_trt(data)[0].cpu()
    x = float(xy[0])
    car.steering = x * steering_gain.value + steering_offset.value
    
    if log_button.value:
        image_path = os.path.join(LOG_DIRECTORY, '%012d_%03d.jpg' % (count, round(50 * x + 50)))
        cv2.imwrite(image_path, image)
        count += 1
    else:
        count = 0

In [ ]:
execute({'new': camera.value})

In [ ]:
camera.observe(execute, names='value')

In [ ]:
camera.unobserve_all()

In [ ]:
!zip -rq circuit_log.zip circuit_log